In [ ]:
import pandas as pd
import json
import shapely
from shapely.geometry import shape, GeometryCollection, Point, Polygon

In [ ]:
with open('seccensales.JSON') as f:
    features = json.load(f)["features"]

In [ ]:
poi_educa = pd.read_csv('educacion_general.csv', sep=',', decimal='.')
poi_lavan = pd.read_csv('lavanderias_general.csv', sep=',', decimal='.')
poi_super = pd.read_csv('supermercado_general.csv', sep=',', decimal='.')
poi_transp = pd.read_csv('transporte_general.csv', sep=',', decimal='.')
poi_gym = pd.read_csv('gimnasios_general.csv', sep=',', decimal='.')
poi_parques = pd.read_csv('parques_general.csv', sep=',', decimal='.')
poi_restau = pd.read_csv('restaurantes_general.csv', sep=',', decimal='.')
poi_bank = pd.read_csv('Bancos_general.csv', sep=',', decimal='.')
poi_estadios = pd.read_csv('estadio_general.csv', sep=',', decimal='.')
poi_hospitales = pd.read_csv('hospital_general.csv', sep=',', decimal='.')

In [ ]:
def remove_duplicates(df, string_type):
  df['duplicator'] = df['name']+df['formatted_address']
  df['duplicator2'] = df['geometry.location.lat']+df['geometry.location.lng']
  df.drop_duplicates(['duplicator'], inplace=True)
  df.drop_duplicates(['duplicator2'], inplace=True)
  df['type']= string_type
  df.drop(['Unnamed: 0','formatted_address','icon','icon_background_color','icon_mask_base_uri','photos','place_id','reference','geometry.viewport.northeast.lat',	'geometry.viewport.northeast.lng',	'geometry.viewport.southwest.lat',	'geometry.viewport.southwest.lng',	'plus_code.compound_code',	'plus_code.global_code',	'opening_hours.open_now',	'duplicator',	'duplicator2'], axis=1, inplace=True)
  return df

In [ ]:
poi_educa = remove_duplicates(poi_educa, 'education')
poi_lavan = remove_duplicates(poi_lavan, 'lavanderias')
poi_super = remove_duplicates(poi_super, 'tiendas')
poi_transp = remove_duplicates(poi_transp, 'transporte')
poi_gym = remove_duplicates(poi_gym, 'gimnasios')
poi_parques = remove_duplicates(poi_parques, 'parques')
poi_restau = remove_duplicates(poi_restau, 'restaurante')
poi_bank = remove_duplicates(poi_bank, 'bancos')
poi_estadios = remove_duplicates(poi_estadios, 'estadios')
poi_hospitales = remove_duplicates(poi_hospitales, 'hospitales')




In [ ]:
poi_df = pd.concat([poi_educa, poi_lavan, poi_super, poi_transp, poi_gym, poi_parques, poi_restau, poi_bank, poi_estadios, poi_hospitales], ignore_index=True)

In [ ]:
def check_polygon(lat,long, infeatures):
  seccensal = 0
  point1 = Point(long, lat)
  for feature in infeatures:
      poly = Polygon(feature["geometry"]["coordinates"][0])
      if poly.contains(point1) == True:
          seccensal = feature["properties"]["coddistsec"]
          break
  return seccensal

In [ ]:
poi_df["coddistsec"] = poi_df.apply(lambda row: check_polygon(row["geometry.location.lat"], row["geometry.location.lng"], features),axis=1)

In [ ]:
poi_df = poi_df[poi_df['coddistsec']!=0.0]

In [ ]:
def divide_string(string_to_analyze):
  new_data = string_to_analyze.split("'")
  if new_data[1] == 'point_of_interest':
    valstr = new_data[3]
  else:
    valstr = new_data[1]
  return valstr

In [ ]:
poi_df["subtypes"] = poi_df.apply(lambda row: divide_string(row["types"]),axis=1)

In [ ]:
poi_df = poi_df[poi_df['permanently_closed']!=True]
poi_df.drop(['permanently_closed','price_level','types', 'business_status'], axis=1, inplace=True)

In [18]:
lavanderias_df = poi_df[poi_df['subtypes']=='laundry']

In [ ]:
poi_df.to_csv(r'poi_tratado.csv')

In [19]:
lavanderias_df.to_csv(r'lavanderias.csv')